I used the DataQuest Kaggle module "Improving your submission" as a resource in my second model iteration.  The first thing it did was introduce the random forest model, so I will start by creating one.

# Imports

In [68]:
from __future__ import division
import pandas as pd
import numpy as np
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, f_classif
import re
import operator

# Load and clean data

In [69]:
def cleanData(data, median_age):
    """
    Take in the raw data and median age from the training data
    and return a cleaned version for use with our model
    """
    # Replace missing ages with the median age (from the training data!)
    data.Age = data.Age.fillna(median_age)

    # Encode male as 0 and female as 1
    data.loc[data.Sex == 'male', 'Sex'] = 0
    data.loc[data.Sex == 'female', 'Sex'] = 1

    # Replace missing port of embarkation with Southampton
    # Emcode Southampton as 0, Cherbourg as 1, and Queenstown as 2
    data.Embarked = data.Embarked.fillna('S')
    data.loc[data.Embarked == 'S', 'Embarked'] = 0
    data.loc[data.Embarked == 'C', 'Embarked'] = 1
    data.loc[data.Embarked == 'Q', 'Embarked'] = 2
    
    # Replace missing fares with the median fare
    data.Fare = data.Fare.fillna(data.Fare.median())
    
    return data

In [70]:
titanic = pd.read_csv('train.csv')
titanic_test = pd.read_csv('test.csv')

titanic = cleanData(titanic, titanic.Age.median())
titanic_test = cleanData(titanic_test, titanic.Age.median())

# Create a random forest model

In [71]:
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
alg = RandomForestClassifier(random_state=1, n_estimators=10, min_samples_split=2, min_samples_leaf=1)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic.Survived, cv=3)
print '{:.3f}% accuracy'.format(scores.mean()*100)

80.022% accuracy


Already it's looking like a random forest model is doing quite a bit better than the logistic regression from iteration 1!

# Tweak model parameters

In [72]:
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=4, min_samples_leaf=2)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic.Survived, cv=3)
print '{:.3f}% accuracy'.format(scores.mean()*100)

81.930% accuracy


Increasing the number of trees used, number of samples required to split, and number of samples required to create a leaf improved the cross validation accuracy by nearly 2% and will help avoid overfitting to the training data.  Let's see how this model does with the test data.

In [73]:
def testModel(alg, train, test, predictors):
    """
    Take in the training data, testing data, and predictors to use
    and return the submission dataframe for a logistic regression model
    """
    # Train the algorithm using all the training data
    alg.fit(train[predictors], train.Survived)

    # Make predictions using the test set.
    predictions = alg.predict(test[predictors])

    # Create a new dataframe with only the columns Kaggle wants from the dataset.
    return pd.DataFrame({
        'PassengerId': test.PassengerId,
        'Survived': predictions
    })

In [74]:
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=4, min_samples_leaf=2)
submission = testModel(alg, titanic, titanic_test, predictors)

# Write submission to a CSV file
submission.to_csv('submission_4.csv', index=False)

75.120 accuracy

Even though the cross validation score was higher than any from the logistic regression model, the random forest model performed the same on the test data as our very first submission.  It seems to me that random forests may be more susceptible to overfitting than the logistic regression, so I will have to be aware that a better cross validation score may not mean that a model will be more accurate with the test data.  I'll deviate from the DataQuest module and try increasing `min_samples_split` and `min_samples_leaf` a bit more to see if that helps.

In [75]:
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic.Survived, cv=3)
print '{:.3f}% accuracy'.format(scores.mean()*100)

81.706% accuracy


As I expected, the cross validation score was a bit lower.  This may indicate that the model has less overfitting and will perform better with the test data.  Let's check.

In [76]:
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)
submission = testModel(alg, titanic, titanic_test, predictors)

# Write submission to a CSV file
submission.to_csv('submission_5.csv', index=False)

77.990% accuracy

This model is the most accurate so far!  I was correct in thinking that this model would be less overfit to the training data and thus do better with the test data.

The next thing the DataQuest module did was create new features.  I'll follow along, create some of my own, then evaluate how much of an impact these features have.

# Create new features

In [77]:
def getTitle(name):
    """
    Take in a name and return the title encoded as an integer, if there is one
    """
    # Use a regular expression to search for a title.
    # Titles always consist of capital and lowercase letters, and end with a period.
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and encode it.
    if title_search:
        title = title_search.group(1)

        # Map each title to an integer.
        # Some titles are very rare, and are compressed into the same codes as other titles.
        title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Dr": 5, "Rev": 6,
                         "Major": 7, "Col": 7, "Mlle": 8, "Mme": 8, "Don": 9, "Lady": 10,
                         "Countess": 10, "Jonkheer": 10, "Sir": 9, "Capt": 7, "Ms": 2}
        try:
            return title_mapping[title]
        except KeyError:
            pass
    return 0

def getFamilyId(row, family_id_mapping):
    """
    Take in a row and the family id map and return the family id and updated map
    Must be run after FamilySize column is created
    """
    # Find the last name by splitting on a comma
    last_name = row.Name.split(",")[0]
    # Create the family id
    family_id = '{}{}'.format(last_name, row.FamilySize)
    # Look up the id in the mapping
    if family_id not in family_id_mapping:
        if len(family_id_mapping) == 0:
            current_id = 1
        else:
            # Get the maximum id from the mapping and add one to it if we don't have an id
            current_id = (max(family_id_mapping.items(), key=operator.itemgetter(1))[1] + 1)
        family_id_mapping[family_id] = current_id
    return family_id_mapping[family_id], family_id_mapping

def createDQFeatures(data, family_id_mapping):
    """
    Add the features from the DataQuest module
    """
    # Generating a familysize column
    data["FamilySize"] = data.SibSp + data.Parch

    # The .apply method generates a new series
    data["NameLength"] = data.Name.apply(lambda x: len(x))

    # Add a title column
    data["Title"] = data.Name.apply(getTitle)

    # Add a family id column
    family_ids = pd.Series(np.zeros((len(data),)))
    # Unlike in DataQuest, I don't want to use global variables
    # Unfortunately, this means I have to iterate over the rows manually
    for i, row in data.iterrows():
        family_id, family_id_mapping = getFamilyId(row, family_id_mapping)
        family_ids[i] = family_id
    # There are a lot of family ids, so we'll compress all of the families under 3 members into one code.
    family_ids[titanic.FamilySize < 3] = -1
    data["FamilyId"] = family_ids

    return data, family_id_mapping

In [78]:
# We want to use the same family ID map in the training and test data, so we need to keep
# track of it (and I don't want to use global variables like the DataQuest module does)
family_id_mapping = {}
titanic, family_id_mapping = createDQFeatures(titanic, family_id_mapping)
titanic_test, family_id_mapping = createDQFeatures(titanic_test, family_id_mapping)

In [79]:
def getTitle2(name):
    """
    Take in a name and return the title encoded as an integer, if there is one
    """
    # Use a regular expression to search for a title.
    # Titles always consist of capital and lowercase letters, and end with a period.
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and encode it.
    if title_search:
        title = title_search.group(1)

        # Map each title to an integer. (Revised groupings by me)
        # 1: Mr (adult male)
        # 2: Miss, Mlle, Ms (young female)
        # 3: Mrs, Mme (adult female)
        # 4: Master (young male)
        # 5: Dr, Rev (special adult)
        # 6: Major, Col (military)
        # 7: Don, Jonkheer, Sir, Capt (very special male)
        # 8: Lady, Countess (very special female)
        title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Dr": 5, "Rev": 5,
                         "Major": 6, "Col": 6, "Mlle": 2, "Mme": 3, "Don": 7, "Lady": 8,
                         "Countess": 8, "Jonkheer": 7, "Sir": 7, "Capt": 7, "Ms": 2}
        try:
            return title_mapping[title]
        except KeyError:
            pass
    return 0

def createMyFeatures(data):
    """
    Add my features
    """
    # Add a flag for children under the age of 9
    data["IsChild"] = data.Age.apply(lambda x: 1 if x < 9 else 0)
    
    # Add another family size column that includes the passenger (like we talked about in class)
    data["FamilySize2"] = data.SibSp + data.Parch + 1
    
    # Add another title column that groups the titles differently
    data["Title2"] = data.Name.apply(getTitle2)

    return data

titanic = createMyFeatures(titanic)
titanic_test = createMyFeatures(titanic_test)

# Select best features to use

In [80]:
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked",
              "FamilySize", "NameLength", "Title", "FamilyId", 
              "IsChild", "FamilySize2", "Title2"]

# Perform feature selection
selector = SelectKBest(f_classif, k='all')
selector.fit(titanic[predictors], titanic["Survived"])

# Get the raw p-values for each feature, and transform from p-values into scores
scores = -np.log10(selector.pvalues_)

for x in sorted(zip(scores, predictors))[::-1]:
    print x

(68.851994252857665, 'Sex')
(32.002333704918456, 'Title2')
(26.983386072106185, 'Title')
(24.595671420777524, 'Pclass')
(23.693190161546514, 'NameLength')
(14.213235141762933, 'Fare')
(5.0206988771330954, 'IsChild')
(2.8513009904508668, 'Embarked')
(1.8716004089590674, 'FamilyId')
(1.8297604290610845, 'Parch')
(1.2776895459668496, 'Age')
(0.53425450244248629, 'SibSp')
(0.20768458341872537, 'FamilySize2')
(0.20768458341872537, 'FamilySize')


In the DataQuest module, they didn't include name length in this part, so I was surprised to see how high it ranked among the predictors.  Maybe the `f_classif` function rated it highly, but it doesn't do much for the random forest.  After seeing some of the data from the name column, it would seem to me that name length would not be that useful for indicating wealth which is what the DataQuest module intended when creating that column.  I was also surprised that title was the only other new feature in the top five.  It looks like my encodings of the titles were slightly better than the ones from the DataQuest module, but including the passenger in the family size had no effect.

Next, I want to see how adding predictors one-by-one impacts cross validation score.

In [81]:
# Predictors in order of importance, leaving out NameLength and using Title2 instead of Title
predictors = ["Sex", "Title2", "Pclass", "Fare", "IsChild", "Embarked", "FamilyId", "FamilySize"]

alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)
for i in range(len(predictors)):
    scores = cross_validation.cross_val_score(alg, titanic[predictors[:i+1]], titanic.Survived, cv=3)
    print '{}: {:.3f}% accuracy'.format(i+1, scores.mean()*100)

1: 78.676% accuracy
2: 79.349% accuracy
3: 77.778% accuracy
4: 81.369% accuracy
5: 80.808% accuracy
6: 82.267% accuracy
7: 82.492% accuracy
8: 82.492% accuracy


There are a couple of times when adding more predictors reduces accuracy, which is surprising to me.  I'll try a few different sets of predictors to see how they do with the test data

In [82]:
predictors = ["Sex", "Title2", "Pclass", "Fare"]
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)
submission = testModel(alg, titanic, titanic_test, predictors)

# Write submission to a CSV file
submission.to_csv('submission_6.csv', index=False)

77.033% accuracy

Using only the top 4 predictors performed worse than the previous submission that used none of the added features.  This isn't too surprising since it had a slighly lower cross validation score.

In [83]:
predictors = ["Sex", "Title2", "Pclass", "Fare", "IsChild", "Embarked"]
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)
submission = testModel(alg, titanic, titanic_test, predictors)

# Write submission to a CSV file
submission.to_csv('submission_7.csv', index=False)

75.598% accuracy

Adding the next 2 predictors decreased the accuracy by over 1%.  Judging by the cross validation scores, this model should have done better.  This may be an indication that the model is overfitting.

In [84]:
predictors = ["Sex", "Title2", "Pclass", "Fare", "IsChild", "Embarked", "FamilyId"]
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)
submission = testModel(alg, titanic, titanic_test, predictors)

# Write submission to a CSV file
submission.to_csv('submission_8.csv', index=False)

76.077% accuracy

Adding the family ID improved performance slightly, but it is still worse than just using the top 4 predictors.  It seems like using fewer predictors helps prevent overfitting and allows the model to perform better with the test data.

So far, none of the models that use the new features have improved with the test data.  So, I want to see how a model does that _only_ uses the new features (except for name length).

In [85]:
predictors = ["Title2", "IsChild", "FamilyId", "FamilySize"]
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic.Survived, cv=3)
print '{:.3f}% accuracy'.format(scores.mean()*100)

82.267% accuracy


In [86]:
predictors = ["Title2", "IsChild", "FamilyId", "FamilySize"]
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)
submission = testModel(alg, titanic, titanic_test, predictors)

# Write submission to a CSV file
submission.to_csv('submission_9.csv', index=False)

78.469% accuracy

This improved our best performance!  It is interesting that the top 4 predictors according to `SelectKBest` and `f_classif` were not as good as the 4 added predictors.  It seems that using a smaller number of predictors is good for preventing overfitting and `SelectKBest` is not necessarily good at selecting the best predictors.  I'm going to choose a set of predictors based on my intuition while trying to keep the number low.

In [87]:
predictors = ["Sex", "Pclass", "Age", "Title2", "FamilyId"]
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic.Survived, cv=3)
print '{:.3f}% accuracy'.format(scores.mean()*100)

82.941% accuracy


In [88]:
predictors = ["Sex", "Pclass", "Age", "Title2", "FamilyId"]
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)
submission = testModel(alg, titanic, titanic_test, predictors)

# Write submission to a CSV file
submission.to_csv('submission_10.csv', index=False)

74.163% accuracy

This one did quite a bit worse, even though the cross validation score looked good.  I'm guessing that overfitting in the culprit again, prossibly due to using the family ID.  I wonder if replacing it with family size will help.

In [89]:
predictors = ["Sex", "Pclass", "Age", "Title2", "FamilySize"]
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic.Survived, cv=3)
print '{:.3f}% accuracy'.format(scores.mean()*100)

82.492% accuracy


In [90]:
predictors = ["Sex", "Pclass", "Age", "Title2", "FamilySize"]
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)
submission = testModel(alg, titanic, titanic_test, predictors)

# Write submission to a CSV file
submission.to_csv('submission_11.csv', index=False)

74.641% accuracy

This one also did worse than the model using only the added features.  I think it's time to try something other than just choosing different predictors.

# Gradient boosting

Gradient boosting involves training the decision trees one at a time and using the errors to help build the next tree.  Since this can easily lead to overfitting, the DataQuest module suggests limiting the number of trees to 25 and the tree depth to 3.  First, I'll use the predictors that the module uses (with my modified title groupings).

In [91]:
predictors = ["Pclass", "Sex", "Age", "Fare", "Embarked", "FamilySize", "Title2", "FamilyId"]
alg = GradientBoostingClassifier(random_state=1, n_estimators=25, max_depth=3)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic.Survived, cv=3)
print '{:.3f}% accuracy'.format(scores.mean()*100)

82.155% accuracy


In [92]:
predictors = ["Pclass", "Sex", "Age", "Fare", "Embarked", "FamilySize", "Title2", "FamilyId"]
alg = GradientBoostingClassifier(random_state=1, n_estimators=25, max_depth=3)
submission = testModel(alg, titanic, titanic_test, predictors)

# Write submission to a CSV file
submission.to_csv('submission_12.csv', index=False)

80.383% accuracy

The gradient boosting boosted our score to the highest yet!  This was also the smallest difference between cross validation score and test score so far, perhaps indicating that this model did not suffer from overfitting as much as the previous ones.  This is interesting to me since the DataQuest module used a lot of columns as predictors, so now I'm going to try using only the predictors from the best random forest—the created features.

In [93]:
predictors = ["Title2", "IsChild", "FamilyId", "FamilySize"]
alg = GradientBoostingClassifier(random_state=1, n_estimators=25, max_depth=3)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic.Survived, cv=3)
print '{:.3f}% accuracy'.format(scores.mean()*100)

83.165% accuracy


In [94]:
predictors = ["Title2", "IsChild", "FamilyId", "FamilySize"]
alg = GradientBoostingClassifier(random_state=1, n_estimators=25, max_depth=3)
submission = testModel(alg, titanic, titanic_test, predictors)

# Write submission to a CSV file
submission.to_csv('submission_13.csv', index=False)

77.512% accuracy

This one didn't do as well, and had a large difference between cross validation score and test score.  I guess with this model, using a large number of predictors can work out pretty well.

# Future explorations

The next thing that the DataQuest module did was use both gradient boosting and a logistic regression as an ensemble.  However, their accuracy with the test data was slightly (1 or 2 predictions) lower than what I got with the gradient boosting alone, so I'm not sure if this would be worth pursuing.  With both the random forest and gradient boosting, I would be interested in looking into a better way of choosing which features to use as predictors.  `SelectKBest` didn't seem to do a great job, and my intuitions often did worse than I expected.  With the way Kaggle is set up, it's impossible to do an exhaustive search using the test data, and as we've seen in this iteration, higher cross validation scores do not necessarily mean better test accuracy.  Is there a good way to search across features to find the best ones to use as predictors?